In [1]:
import torch
import random
import pandas as pd
import numpy as np

import main_regression as mr

# Set Seed

In [2]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

# Set Config

In [3]:
# Case 1. LSTM model (w/o data representation)
config1 = {
        'model': 'LSTM', # Regression에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC', 'DARNN} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/lstm.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 27,  # 데이터의 변수 개수, int
            'timestep' : 10, # timestep = window_size
            'shift_size': 1, # shift 정도, int
            'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
            'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
            'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bidirectional': True,  # 모델의 양방향성 여부, bool(default: True)
            'num_epochs': 150,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 64,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
            'device': 'cuda',  # 학습 환경, ["cuda", "cpu"] 중 선택
            'need_yhist' : False
        }
}

# Case 2. GRU model (w/o data representation)
config2 = {
        'model': 'GRU', # Regression에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC', 'DARNN} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/gru.pt',  # 학습 완료 모델 저장 경로
        'with_representation' : False, # representation 유무, bool (defeault: False)
        'parameter': {
            'input_size': 27,  # 데이터의 변수 개수, int
            'timestep' : 10, # timestep = window_size
            'shift_size': 1, # shift 정도, int
            'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
            'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
            'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bidirectional': True,  # 모델의 양방향성 여부, bool(default: True)
            'num_epochs': 150,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 64,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
            'device': 'cuda',  # 학습 환경, ["cuda", "cpu"] 중 선택
            'need_yhist' : False
        }
}

# Case 3. CNN_1D model (w/o data representation)
config3 = {
        'model': 'CNN_1D', # Regression에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC', 'DARNN} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/cnn_1d.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 27,  # 데이터의 변수 개수, int
            'timestep' : 10, # timestep = window_size
            'shift_size': 1, # shift 정도, int
            'seq_len': 10,  # 데이터의 시간 길이, int
            'output_channels': 64, # convolution layer의 output channel, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
            'kernel_size': 3, # convolutional layer의 filter 크기, int(default: 3, 범위: 3 이상, 홀수로 설정 권장)
            'stride': 1, # convolution layer의 stride 크기, int(default: 1, 범위: 1 이상)
            'padding': 0, # padding 크기, int(default: 0, 범위: 0 이상)
            'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'num_epochs': 150,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 64,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda',  # 학습 환경, ["cuda", "cpu"] 중 선택
            'need_yhist' : False
        }
}

# Case 4. DA-RNN model (w/o data representation)
config4 = {
        'model': 'LSTM_FCNs', # Regression에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC', 'DARNN} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/lstm_fcn.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 27,  # 데이터의 변수 개수, int
            'timestep' : 10, # timestep = window_size
            'shift_size': 1, # shift 정도, int
            'num_layers': 1,  # recurrent layers의 수, int(default: 1, 범위: 1 이상)
            'lstm_drop_out': 0.4, # LSTM dropout 확률, float(default: 0.4, 범위: 0 이상 1 이하)
            'fc_drop_out': 0.1, # FC dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'num_epochs': 150, # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 64,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda',  # 학습 환경, ["cuda", "cpu"] 중 선택
            'need_yhist' : False
        }
}

# Case 5. DARNN model (w/o data representation)
config5 = {
        'model': 'DARNN', # Regression에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC', 'DARNN} 중 택 1
        'training': True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        'best_model_path': './ckpt/darnn.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 27,  # 데이터의 변수 개수, int
            'encoder_hidden_size': 64, # Encoder hidden state의 차원, int(default: 64, 범위: 1 이상)
            'decoder_hidden_size': 64, # Decoder hidden state의 차원, int(default: 64, 범위: 1 이상)
            'timestep': 16, # timestep의 크기, int(default: 16, 범위: 1이상),
            'shift_size' : 1, # Slicing 시 shift 크기
            'encoder_stateful': False, # Encoder의 Stateful 사용여부, bool(default: False)
            'decoder_stateful': False, # Decoder의 Stateful 사용여부, bool(default: False)
            'num_epochs': 150,  # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 64,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda',  # 학습 환경, ["cuda", "cpu"] 중 선택
            'need_yhist': True
        }
}

# Case 6. fully-connected layers (w/ data representation)
config6 = {
        'model': 'FC', # Regression에 활용할 알고리즘 정의, {'LSTM', 'GRU', 'CNN_1D', 'LSTM_FCNs', 'FC', 'DARNN} 중 택 1
        "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
        "best_model_path": './ckpt/fc.pt',  # 학습 완료 모델 저장 경로
        'parameter': {
            'input_size': 64,  # 데이터의 변수 개수(representation 차원), int
            'timestep' : 1, # timestep = window_size
            'shift_size': 1, # shift 정도, int
            'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bias': True, # bias 사용 여부, bool(default: True)
            'num_epochs': 150, # 학습 epoch 횟수, int(default: 150, 범위: 1 이상)
            'batch_size': 32,  # batch 크기, int(default: 64, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
            'lr': 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
            'device': 'cuda',  # 학습 환경, ["cuda", "cpu"] 중 선택
            'need_yhist' : False
        }
}


# Load data

In [4]:
# raw time seires data for regression
train = pd.read_csv('./data/train_data.csv')
test = pd.read_csv('./data/test_data.csv')

train = train.drop('date', axis=1)
test = test.drop('date', axis=1)

train_x = train.drop('Appliances', axis = 1)
train_y = train['Appliances']

test_x = test.drop('Appliances', axis = 1)
test_y = test['Appliances']

train_data = {'x': train_x, 'y': train_y}
test_data = {'x': test_x, 'y': test_y}

print(train_x.shape)  #shape : (num_of_instance x representation_dims) = (13866, 27)
print(train_y.shape) #shape : (num_of_instance) = (13866, )
print(test_x.shape)  #shape : (num_of_instance x representation_dims) = (5869, 27)
print(test_y.shape)  #shape : (num_of_instance) = (5869, )

(13866, 27)
(13866,)
(5869, 27)
(5869,)


# Training Model

In [5]:
# Case 1. LSTM model (w/o data representation)
config = config1
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

y_true, pred, mse, r2 = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측
print(f'test Loss: {np.round(mse,5)} and R2: {np.round(r2,5)}')

Start training model

Epoch 1/150
train Loss: 21109.8459
val Loss: 19662.5854

Epoch 10/150
train Loss: 17096.3751
val Loss: 15943.8629

Epoch 20/150
train Loss: 14810.4974
val Loss: 13671.1379

Epoch 30/150
train Loss: 13273.7881
val Loss: 12137.5864

Epoch 40/150
train Loss: 12338.1913
val Loss: 11203.1128

Epoch 50/150
train Loss: 11881.1846
val Loss: 10744.6356

Epoch 60/150
train Loss: 11735.8969
val Loss: 10595.2767

Epoch 70/150
train Loss: 11703.7689
val Loss: 10550.6393

Epoch 80/150
train Loss: 10797.0913
val Loss: 10217.3282

Epoch 90/150
train Loss: 10503.9071
val Loss: 10114.9095

Epoch 100/150
train Loss: 10218.2018
val Loss: 9974.0889

Epoch 110/150
train Loss: 9985.1052
val Loss: 9952.9005

Epoch 120/150
train Loss: 9837.4271
val Loss: 10052.6819

Epoch 130/150
train Loss: 9684.4925
val Loss: 10084.6856

Epoch 140/150
train Loss: 9454.1828
val Loss: 10001.7870

Epoch 150/150
train Loss: 9277.9204
val Loss: 10147.0717

Training complete in 1m 37s
Best val Loss: 9952.9005

In [6]:
# Case 2. GRU (w/o data representation)
config = config2
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

y_true, pred, mse, r2 = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측
print(f'test Loss: {np.round(mse,5)} and R2: {np.round(r2,5)}')

Start training model

Epoch 1/150
train Loss: 20953.5274
val Loss: 19423.9751

Epoch 10/150
train Loss: 16861.7194
val Loss: 15697.0454

Epoch 20/150
train Loss: 14446.9247
val Loss: 13298.1671

Epoch 30/150
train Loss: 12921.4872
val Loss: 11780.3374

Epoch 40/150
train Loss: 12096.7693
val Loss: 10958.9671

Epoch 50/150
train Loss: 11780.5811
val Loss: 10641.2651

Epoch 60/150
train Loss: 11186.6239
val Loss: 10208.4844

Epoch 70/150
train Loss: 10613.9882
val Loss: 10015.3815

Epoch 80/150
train Loss: 10217.9469
val Loss: 10077.3371

Epoch 90/150
train Loss: 10012.2731
val Loss: 10235.9645

Epoch 100/150
train Loss: 9766.5728
val Loss: 10465.0258

Epoch 110/150
train Loss: 9580.0131
val Loss: 10816.8436

Epoch 120/150
train Loss: 9320.5673
val Loss: 10894.6368

Epoch 130/150
train Loss: 9078.9983
val Loss: 11106.7284

Epoch 140/150
train Loss: 8935.8988
val Loss: 11508.9812

Epoch 150/150
train Loss: 8831.4899
val Loss: 11279.4516

Training complete in 1m 30s
Best val Loss: 9985.417

In [7]:
# Case 3. CNN_1D (w/o data representation)
config = config3
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

y_true, pred, mse, r2 = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측
print(f'test Loss: {np.round(mse,5)} and R2: {np.round(r2,5)}')

Start training model

Epoch 1/150
train Loss: 13304.8375
val Loss: 10569.9333

Epoch 10/150
train Loss: 11400.6595
val Loss: 10201.7438

Epoch 20/150
train Loss: 11026.7111
val Loss: 10176.2793

Epoch 30/150
train Loss: 10899.6038
val Loss: 10025.8382

Epoch 40/150
train Loss: 10803.6478
val Loss: 10036.5983

Epoch 50/150
train Loss: 10725.9903
val Loss: 10042.6370

Epoch 60/150
train Loss: 10630.0849
val Loss: 10142.4324

Epoch 70/150
train Loss: 10531.2082
val Loss: 10077.1398

Epoch 80/150
train Loss: 10460.5885
val Loss: 10146.8854

Epoch 90/150
train Loss: 10386.8843
val Loss: 10136.3393

Epoch 100/150
train Loss: 10308.1372
val Loss: 10177.3934

Epoch 110/150
train Loss: 10235.1319
val Loss: 10099.3595

Epoch 120/150
train Loss: 10124.5098
val Loss: 10108.7842

Epoch 130/150
train Loss: 10061.6208
val Loss: 10488.3482

Epoch 140/150
train Loss: 10005.0993
val Loss: 10090.5710

Epoch 150/150
train Loss: 9979.6457
val Loss: 10040.3204

Training complete in 0m 52s
Best val Loss: 991

In [8]:
# Case 4. LSTM_FCNs (w/o data representation)
config = config4
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

y_true, pred, mse, r2 = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측
print(f'test Loss: {np.round(mse,5)} and R2: {np.round(r2,5)}')

Start training model

Epoch 1/150
train Loss: 21383.8604
val Loss: 20345.9351

Epoch 10/150
train Loss: 20296.7272
val Loss: 19156.4329

Epoch 20/150
train Loss: 18529.9046
val Loss: 14832.9598

Epoch 30/150
train Loss: 16582.0329
val Loss: 18094.7040

Epoch 40/150
train Loss: 14538.8215
val Loss: 17244.9707

Epoch 50/150
train Loss: 12363.4452
val Loss: 13221.4332

Epoch 60/150
train Loss: 10231.5178
val Loss: 14333.3678

Epoch 70/150
train Loss: 8365.0164
val Loss: 11878.6040

Epoch 80/150
train Loss: 6664.9288
val Loss: 13083.2043

Epoch 90/150
train Loss: 5329.5436
val Loss: 13817.8367

Epoch 100/150
train Loss: 4481.9931
val Loss: 9945.4765

Epoch 110/150
train Loss: 3541.9833
val Loss: 10488.4278

Epoch 120/150
train Loss: 3357.8991
val Loss: 10350.3869

Epoch 130/150
train Loss: 2862.8963
val Loss: 12498.7640

Epoch 140/150
train Loss: 2999.9153
val Loss: 9490.4170

Epoch 150/150
train Loss: 2637.0754
val Loss: 9862.0899

Training complete in 1m 58s
Best val Loss: 9353.963914

S

In [ ]:
# Case 5. DARNN model (w/o data representation)
config = config5
data_reg = mr.Regression(config, train_data, test_data)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

y_true, pred, mse, r2 = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측
print(f'test Loss: {np.round(mse,5)} and R2: {np.round(r2,5)}')

In [10]:
train_repr = pd.read_csv('./data/train_repr.csv')
test_repr = pd.read_csv('./data/test_repr.csv')

train_x_repr = np.array(train_repr)
test_x_repr = np.array(test_repr)

train_y_repr = np.array(train_y[9:])
test_y_repr = np.array(test_y[9:])

train_data_repr = {'x': train_x_repr, 'y': train_y_repr}
test_data_repr = {'x': test_x_repr, 'y': test_y_repr}

In [12]:
# Case 6. fully-connected layers (w/ data representation)

# raw time seires data for regression
config = config6
data_reg = mr.Regression(config, train_data_repr, test_data_repr, use_representation = True)
model = data_reg.build_model()  # 모델 구축

if config["training"]:
    best_model = data_reg.train_model(model)  # 모델 학습
    data_reg.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

y_true, pred, mse, r2 = data_reg.pred_data(model, best_model_path=config["best_model_path"])  # 예측
print(f'test Loss: {np.round(mse,5)} and R2: {np.round(r2,5)}')

Start training model

Epoch 1/150
train Loss: 21387.8319
val Loss: 20262.9520

Epoch 10/150
train Loss: 13707.9351
val Loss: 12050.3008

Epoch 20/150
train Loss: 11516.6349
val Loss: 10256.0511

Epoch 30/150
train Loss: 11084.4216
val Loss: 10029.6628

Epoch 40/150
train Loss: 10829.8577
val Loss: 9878.0595

Epoch 50/150
train Loss: 10673.4709
val Loss: 9769.3786

Epoch 60/150
train Loss: 10550.0279
val Loss: 9697.7771

Epoch 70/150
train Loss: 10464.0775
val Loss: 9650.9635

Epoch 80/150
train Loss: 10406.4277
val Loss: 9628.1403

Epoch 90/150
train Loss: 10291.7530
val Loss: 9610.4862

Epoch 100/150
train Loss: 10278.3956
val Loss: 9608.5635

Epoch 110/150
train Loss: 10275.6640
val Loss: 9613.1159

Epoch 120/150
train Loss: 10153.4856
val Loss: 9630.4084

Epoch 130/150
train Loss: 10194.2315
val Loss: 9640.5241

Epoch 140/150
train Loss: 10119.7021
val Loss: 9651.8129

Epoch 150/150
train Loss: 10088.0426
val Loss: 9666.9644

Training complete in 1m 9s
Best val Loss: 9606.644765

St